# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
resourceUrl='../output_data/cities.csv'
cities_df=pd.read_csv(resourceUrl,index_col=0)
cities_df.head(2)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,mataura,-46.193952,168.865478,8.60,95,61,0.76,New Zealand,1662708397
1,saskylakh,71.954308,114.119835,4.32,97,100,5.76,Sakha Republic,1662708398


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
cities_df = cities_df.dropna()

locations = cities_df[["Lat", "Lng"]].astype(float)
humidity = cities_df["Humidity"].astype(float)

In [4]:

figure_layout = {
    'width': '800px',
    'height': '450px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=90,opacity=0.6,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
perfect_df=cities_df.loc[(cities_df['Max Temp'] < 27) & (cities_df['Max Temp'] > 21) & \
     (cities_df['Wind Speed'] < 4.5 ) &
     (cities_df['Cloudiness'] == 0) &
     (cities_df['Humidity'] < 50) 
     ]
print(f'{len(perfect_df)} cities left.')
perfect_df

10 cities left.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,tsihombe,-25.316847,45.486309,25.12,44,0,3.82,Androy,1662708408
14,capitan bado,-23.258548,-55.541359,22.03,47,0,3.62,Paraguay,1662708411
72,sidvokodvo,-26.632129,31.421332,23.03,31,0,0.87,Eswatini,1662708483
128,cedar city,37.677477,-113.061893,22.44,34,0,3.60,Utah,1662708549
191,barra do garcas,-15.891620,-52.261883,22.26,34,0,2.75,Barra do Garças,1662708647
255,mulanje,-16.025234,35.508314,25.16,42,0,2.03,Southern Region,1662708734
289,kazalinsk,45.764056,62.099913,23.56,17,0,1.75,Kyzylorda Province,1662708778
333,bindura,-17.304100,31.327439,23.95,30,0,2.11,Mashonaland Central Province,1662708830
370,umzimvubu,-30.778175,28.952865,21.61,32,0,1.33,Eastern Cape,1662708875
439,saurimo,-9.661217,20.390460,21.18,49,0,2.57,Lunda Sul Province,1662708949


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
from pprint import pprint

# Mulanje need 10000m radius and Umzimvubu need 20000m radius to get a hotel
place_url='https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
params={
    'types':'lodging',
    'radius':20000,
    'keyword':'hotel',
    'key':g_key
}

hotel_df=perfect_df[['City','Country','Lat','Lng']].copy()
hotel_df

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # make request
    hotel_js = requests.get(place_url, params=params).json()

    try:
        hotel_name      =hotel_js['results'][0]['name']
        hotel_address   =hotel_js['results'][0]['vicinity']
        h_lat           =hotel_js['results'][0]['geometry']['location']['lat']
        h_lng           =hotel_js['results'][0]['geometry']['location']['lng']
        hotel_df.loc[index,'Hotel Name']    =hotel_name
        hotel_df.loc[index,'Address']       =hotel_address
        hotel_df.loc[index,'Lat']           =h_lat
        hotel_df.loc[index,'Lng']           =h_lng
    except:
        print(f"{row['City']} has not found any hotel within {params['radius']} metre radius.")
    

hotel_df 

,City,Country,Lat,Lng,Hotel Name,Address
11,tsihombe,Androy,-25.316425,45.488593,Littoral Hôtel,"MFMQ+CCG, Tsiombe"
14,capitan bado,Paraguay,-23.268245,-55.535616,Hotel Suiza,"PFJ7+MQ9, Capitán Bado"
72,sidvokodvo,Eswatini,-26.493433,31.384835,The George Hotel,"Corner Ngwane &, Du Toit Street"
128,cedar city,Utah,37.705146,-112.849792,Cedar Breaks Lodge by Diamond Resorts,"223 Hunter Ridge Dr, Brian Head"
191,barra do garcas,Barra do Garças,-15.879613,-52.311928,Hotel Boa Viagem,Whatsapp: (66 - Saída Para Cuiabá - R. Diamant...
255,mulanje,Southern Region,-16.020486,35.519150,Kara O'mula Country Lodge,"XGH9+RM5, Mulanje"
289,kazalinsk,Kyzylorda Province,45.846317,62.151385,Altyn - Orda,"R5W2+GHC, ул. Жанкожа батыра, Kyzylorda"
333,bindura,Mashonaland Central Province,-17.308051,31.331969,Hala Select Kimberley Hotel,"61 Robert Mugabe Way, Bindura"
370,umzimvubu,Eastern Cape,-30.892996,28.997563,B&B Wolwekraal,"N2 N2, Mount Frere"
439,saurimo,Lunda Sul Province,-9.657113,20.392080,Hotel Kawissa Saurimo,"R. Gov. Martins Soares, Bairro 11 Novembro, Sa..."


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# python 3.10 remove the collections.Iterable, so gmaps will report a error message "module 'collections' has no attribute 'Iterable'".
# Let check the python version and make the collections.Iterable equal collections.abc.Iterable
import sys
if sys.version_info.major >= 3 and sys.version_info.minor >= 10:
    import collections.abc
    collections.Iterable = collections.abc.Iterable

# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations, hover_text='', label='', info_box_content=hotel_info, display_info_box=True)
fig.add_layer(marker_layer)
# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…